In [253]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

In [254]:
train_data = torch.load('../Data/tao_tensors/train_data.pt')
train_label = torch.load('../Data/tao_tensors/train_label.pt')
device= torch.device("cpu")

print(train_data[:10])

tensor([[  80.0000,    3.0000,    7.0000,   -0.0200, -109.4600,   -6.8000,
            0.7000,   81.2365,   26.1400,   26.2400],
        [  80.0000,    3.0000,    8.0000,   -0.0200, -109.4600,   -4.9000,
            1.1000,   81.2365,   25.6600,   25.9700],
        [  80.0000,    3.0000,    9.0000,   -0.0200, -109.4600,   -4.5000,
            2.2000,   81.2365,   25.6900,   25.2800],
        [  80.0000,    3.0000,   10.0000,   -0.0200, -109.4600,   -3.8000,
            1.9000,   81.2365,   25.5700,   24.3100],
        [  80.0000,    3.0000,   11.0000,   -0.0200, -109.4600,   -4.2000,
            1.5000,   81.2365,   25.3000,   23.1900],
        [  80.0000,    3.0000,   12.0000,   -0.0200, -109.4600,   -4.4000,
            0.3000,   81.2365,   24.7200,   23.6400],
        [  80.0000,    3.0000,   13.0000,   -0.0200, -109.4600,   -3.2000,
            0.1000,   81.2365,   24.6600,   24.3400],
        [  80.0000,    3.0000,   14.0000,   -0.0200, -109.4600,   -3.1000,
            0.6000,   

In [255]:
data = pd.read_csv('../Cleaning-the-data/tao-cleaned.csv')
input_labels = ['Zonal_Winds', 'Meridional_Winds', 'Humidity', 'Air_Temp', 'Sea_Surface_Temp']
output_label = 'Sea_Surface_Temp'

# Normalize
for col in input_labels:
    data[col] = (data[col] - data[col].mean()) / data[col].std()

# Take needed columns and a subset for speed
label = data[output_label].values
data = data[input_labels].values
train_size, test_size = 10000, 10000

train_data, test_data = data[:train_size], data[:train_size]
train_label, test_label = label[:train_size], label[:train_size]

# 1 bunch of 10.000 words of vocab 5
train_data = torch.Tensor(train_data).view(train_size, 5, 1)
train_label = torch.Tensor(train_label).view(train_size, 1, 1)
test_data = torch.Tensor(test_data).view(test_size, 5, 1)

In [256]:
bs = 1
vocab_size = 5

In [257]:
net = nn.RNN(5, 1, num_layers=1, nonlinearity='relu')

In [259]:
_, pred = net(train_data[:10].view(10, bs, 5))
actual = train_label[10].view(1, bs, 1)

loss = criterion(pred, actual)

print(pred, actual, loss)


tensor([[[0.]]], grad_fn=<ViewBackward>) tensor([[[-0.5444]]]) tensor(0.2964, grad_fn=<MseLossBackward>)


In [260]:
start=time.time()
criterion = torch.nn.MSELoss()
my_lr = 0.1
optimizer = torch.optim.SGD( net.parameters() , lr=my_lr )
seq_length = 20

for epoch in range(50):
    # set the running quatities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0
        
    for count in range( 0 , 10000-seq_length ,  seq_length):
        
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[count: count+seq_length].view(seq_length, bs, 5)
        minibatch_label = train_label[seq_length].view(1, bs, 1)     
        
        # send them to the gpu
        minibatch_data = minibatch_data.to(device)
        minibatch_label = minibatch_label.to(device)
                       
        # forward the minibatch through the net    
        # print(minibatch_data)
        _, score = net(minibatch_data)
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(score, minibatch_label)
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()
#         utils.normalize_gradient(net);
        optimizer.step()
        
        
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed, '\t loss=',  math.exp(total_loss))



epoch= 0 	 time= 0.8681418895721436 	 loss= 1.000743260415782

epoch= 1 	 time= 1.853024959564209 	 loss= 1.0

epoch= 2 	 time= 2.824151039123535 	 loss= 1.0

epoch= 3 	 time= 3.6889808177948 	 loss= 1.0

epoch= 4 	 time= 4.51352071762085 	 loss= 1.0

epoch= 5 	 time= 5.347130060195923 	 loss= 1.0

epoch= 6 	 time= 6.173182964324951 	 loss= 1.0

epoch= 7 	 time= 7.042313814163208 	 loss= 1.0

epoch= 8 	 time= 7.954385042190552 	 loss= 1.0

epoch= 9 	 time= 8.859261751174927 	 loss= 1.0


KeyboardInterrupt: 